<a href="https://colab.research.google.com/github/MadhaviMRao/Machine-Learning/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip
!pip install bert-serving-server --no-deps

--2019-11-29 09:12:14--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.15.128, 2a00:1450:400c:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.15.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   240MB/s    in 1.6s    

2019-11-29 09:12:21 (240 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  
     |█████████

In [0]:
import os
import tensorflow as tf
sesh = tf.InteractiveSession()

from bert_serving.server.graph import optimize_graph
from bert_serving.server.helper import get_args_parser

# input dir
MODEL_DIR = '/content/uncased_L-12_H-768_A-12' #@param {type:"string"}
# output dir
GRAPH_DIR = '/content/graph/' #@param {type:"string"}
# output filename
GRAPH_OUT = 'extractor.pbtxt' #@param {type:"string"}

POOL_STRAT = 'REDUCE_MEAN' #@param ['REDUCE_MEAN', 'REDUCE_MAX', "NONE"]
POOL_LAYER = '-2' #@param {type:"string"}
SEQ_LEN = '256' #@param {type:"string"}

tf.gfile.MkDir(GRAPH_DIR)


parser = get_args_parser()
carg = parser.parse_args(args=['-model_dir', MODEL_DIR,
                               '-graph_tmp_dir', GRAPH_DIR,
                               '-max_seq_len', str(SEQ_LEN),
                               '-pooling_layer', str(POOL_LAYER),
                               '-pooling_strategy', POOL_STRAT])

tmp_name, config = optimize_graph(carg)
graph_fout = os.path.join(GRAPH_DIR, GRAPH_OUT)

tf.gfile.Rename(
    tmp_name,
    graph_fout,
    overwrite=True
)
print("\nSerialized graph to {}".format(graph_fout))

I:GRAPHOPT:[gra:opt: 52]:model config: /content/uncased_L-12_H-768_A-12/bert_config.json
I:GRAPHOPT:[gra:opt: 55]:checkpoint: /content/uncased_L-12_H-768_A-12/bert_model.ckpt
I:GRAPHOPT:[gra:opt: 59]:build graph...
I:GRAPHOPT:[gra:opt:129]:load parameters from checkpoint...
I:GRAPHOPT:[gra:opt:133]:optimize...
I:GRAPHOPT:[gra:opt:141]:freeze...
I:GRAPHOPT:[gra:opt:146]:write graph to a tmp file: /content/graph/tmp5f9ew5ko



Serialized graph to /content/graph/extractor.pbtxt


In [0]:
import logging
import numpy as np

from tensorflow.python.estimator.estimator import Estimator
from tensorflow.python.estimator.run_config import RunConfig
from tensorflow.python.estimator.model_fn import EstimatorSpec
from tensorflow.keras.utils import Progbar

from bert_serving.server.bert.tokenization import FullTokenizer
from bert_serving.server.bert.extract_features import convert_lst_to_features


log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)
log.handlers = []

In [0]:
GRAPH_PATH = "/content/graph/extractor.pbtxt" #@param {type:"string"}
VOCAB_PATH = "/content/uncased_L-12_H-768_A-12/vocab.txt" #@param {type:"string"}

SEQ_LEN = 256 #@param {type:"integer"}

In [0]:
INPUT_NAMES = ['input_ids', 'input_mask', 'input_type_ids']
bert_tokenizer = FullTokenizer(VOCAB_PATH)

def build_feed_dict(texts):
    
    text_features = list(convert_lst_to_features(
        texts, SEQ_LEN, SEQ_LEN, 
        bert_tokenizer, log, False, False))

    target_shape = (len(texts), -1)

    feed_dict = {}
    for iname in INPUT_NAMES:
        features_i = np.array([getattr(f, iname) for f in text_features])
        features_i = features_i.reshape(target_shape).astype("int32")
        feed_dict[iname] = features_i

    return feed_dict

In [0]:
def build_input_fn(container):
    
    def gen():
        while True:
          try:
            yield build_feed_dict(container.get())
          except:
            yield build_feed_dict(container.get())

    def input_fn():
        return tf.data.Dataset.from_generator(
            gen,
            output_types={iname: tf.int32 for iname in INPUT_NAMES},
            output_shapes={iname: (None, None) for iname in INPUT_NAMES})
    return input_fn

class DataContainer:
  def __init__(self):
    self._texts = None
  
  def set(self, texts):
    if type(texts) is str:
      texts = [texts]
    self._texts = texts
    
  def get(self):
    return self._texts

In [0]:
def model_fn(features, mode):
    with tf.gfile.GFile(GRAPH_PATH, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    output = tf.import_graph_def(graph_def,
                                 input_map={k + ':0': features[k] for k in INPUT_NAMES},
                                 return_elements=['final_encodes:0'])

    return EstimatorSpec(mode=mode, predictions={'output': output[0]})
  
estimator = Estimator(model_fn=model_fn)

Using temporary folder as model directory: /tmp/tmpf8ame43x


In [0]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def build_vectorizer(_estimator, _input_fn_builder, batch_size=128):
  container = DataContainer()
  predict_fn = _estimator.predict(_input_fn_builder(container), yield_single_examples=False)
  
  def vectorize(text, verbose=False):
    x = []
    bar = Progbar(len(text))
    for text_batch in batch(text, batch_size):
      container.set(text_batch)
      x.append(next(predict_fn)['output'])
      if verbose:
        bar.add(len(text_batch))
      
    r = np.vstack(x)
    return r
  
  return vectorize

In [0]:
bert_vectorizer = build_vectorizer(estimator, build_input_fn)

In [0]:
bert_vectorizer

<function __main__.build_vectorizer.<locals>.vectorize>

building a nearest neighbour search engine

In [0]:
class L2Retriever:
    def __init__(self, dim, top_k=3, use_norm=False, use_gpu=True):
        print("l2 0.1----")
        self.dim = dim
        self.top_k = top_k
        self.use_norm = use_norm
        config = tf.ConfigProto(
            device_count={'GPU': (1 if use_gpu else 0)}
        )
        self.session = tf.Session(config=config)
        print("l2 0.2----")
        self.norm = None
        print("l2 0.3----")
        self.query = tf.placeholder("float", [self.dim])
        print("l2 0.4----")
        self.kbase = tf.placeholder("float", [None, self.dim])
        print("l2 1----")
        self.build_graph()
        print("l2 2----")

    def build_graph(self):
        print("l2 3----")

      
        if self.use_norm:
            self.norm = tf.placeholder("float", [None, 1])
        print("l2 4----")

        distance = dot_l2_distances(self.kbase, self.query, self.norm)
        top_neg_dists, top_indices = tf.math.top_k(tf.negative(distance), k=self.top_k)
        top_dists = tf.negative(top_neg_dists)

        self.top_distances = top_dists
        self.top_indices = top_indices

    def predict(self, kbase, query, norm=None):
        print("l2 5----")  
        query = np.squeeze(query)
        feed_dict = {self.query: query, self.kbase: kbase}
        if self.use_norm:
          feed_dict[self.norm] = norm
        
        I, D = self.session.run([self.top_indices, self.top_distances],
                                feed_dict=feed_dict)
        
        return I, D
      
def dot_l2_distances(kbase, query, norm=None):
    query = tf.reshape(query, (1, -1))
    
    if norm is None:
      XX = tf.keras.backend.batch_dot(kbase, kbase, axes=1)
    else:
      XX = norm
    YY = tf.matmul(query, tf.transpose(query))
    XY = tf.matmul(kbase, tf.transpose(query))
    
    distance = XX - 2 * XY + YY
    distance = tf.sqrt(tf.reshape(distance, (-1,)))
    
    return distance

In [0]:
from google.colab import files
import io

#uploaded = files.upload()

In [0]:
#for fn in uploaded.keys():
 # print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [0]:
#print(uploaded.keys())

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# New Section

In [0]:
f= open('/content/gdrive/My Drive/Colab Notebooks/txt1.txt','r')
txt1=f.readlines()
f= open('/content/gdrive/My Drive/Colab Notebooks/txt2.txt','r')
txt2=f.readlines()
f= open('/content/gdrive/My Drive/Colab Notebooks/txt3.txt','r')
txt3=f.readlines()
f= open('/content/gdrive/My Drive/Colab Notebooks/txt4.txt','r')
txt4=f.readlines()
f= open('/content/gdrive/My Drive/Colab Notebooks/txt5.txt','r')
txt5=f.readlines()
f= open('/content/gdrive/My Drive/Colab Notebooks/txt6.txt','r')
txt6=f.readlines()
f= open('/content/gdrive/My Drive/Colab Notebooks/physics.txt','r')
txt7=f.readlines()
#/content/gdrive/My Drive/Colab Notebooks/txt1.txt



In [0]:
txt6

['A typical AI analyzes its environment and takes actions that maximize its chance of success.[1] An AI\'s intended utility function (or goal) can be simple ("1 if the AI wins a game of Go, 0 otherwise") or complex ("Do mathematically similar actions to the ones succeeded in the past"). Goals can be explicitly defined, or induced. If the AI is programmed for "reinforcement learning", goals can be implicitly induced by rewarding some types of behavior or punishing others.[a] Alternatively, an evolutionary system can induce goals by using a "fitness function" to mutate and preferentially replicate high-scoring AI systems, similarly to how animals evolved to innately desire certain goals such as finding food.[59] Some AI systems, such as nearest-neighbor, instead of reason by analogy, these systems are not generally given goals, except to the degree that goals are implicit in their training data.[60] Such systems can still be benchmarked if the non-goal system is framed as a system whose 

In [0]:
X=[]
title=[]
text=[]


list_txt=[txt1,txt2,txt3,txt4,txt5,txt6,txt7]

print(list_txt)
for doc in list_txt:
  tx=""
  t=""
  c=True
  for line in doc:
    if line.startswith('--'):
      if c==False:
        continue
      c=False
      t=line.split('--')[1].replace('\n','')
    else:
      tx=tx+line.replace('\n','').strip()
  title.append(t)
  text.append(t+". "+tx)

print("title ------",len(title))
print("text ---------", len(text))
print("title is---------", title)
print("text is---------", text)



[["-- ECB's Liikanen says global economy more uncertain\n", '-- \n', '-- Tue Aug 16, 2011 4:07am EDT\n', '-- http://www.reuters.com/article/2011/08/16/us-ecb-liikanen-idUSTRE77F1CB20110816\n', '\n', ' \n', '\n', ' TURKU, Finland  (Reuters) - The global economy has moved in an alarming and uncertain direction in recent weeks, ECB Governing Council member Erkki Liikanen said on Tuesday. \n', '\n', ' "The situation in the international economy has again moved in an alarming direction in last weeks. There is more uncertainty about economic growth than before," he said in a speech at a seminar in Turku, Finland. Liikanen said the European Central Bank has taken decisive measures to ensure the transmission of monetary policy. "However, monetary policy can\'t alone solve tension in the financial markets as the background is excessive debt in public finances. It requires action by governments and parliaments," he said. (Reporting by  Jussi Rosendahl )'], ['-- Exclusive: ECB mulls ultra-long lo

In [0]:
X=text
X_vect = bert_vectorizer(X, verbose=True)

7/7 [==============================] - 0s 53ms/step


In [0]:
X_vect

array([[-0.49904892, -0.3610586 ,  0.42659786, ..., -0.6025254 ,
        -0.17197576, -0.13347694],
       [-0.46920526, -0.22283776,  0.5271894 , ..., -0.47161454,
         0.02238406,  0.13185011],
       [-0.42871392, -0.3538573 ,  0.6045447 , ..., -0.3782158 ,
         0.1800992 , -0.03583926],
       ...,
       [-0.41836393, -0.35163736,  0.50353813, ..., -0.37041453,
        -0.01689446,  0.06837612],
       [-0.21314053,  0.09226297, -0.08690625, ..., -0.42807505,
        -0.14325789,  0.8496102 ],
       [-0.39047194, -0.06576734,  0.56919116, ..., -0.36148703,
         0.03652759,  0.32127047]], dtype=float32)

In [0]:
def buildMovieRecommender(movie_names, vectorized_plots, top_k=10):
  retriever = L2Retriever(vectorized_plots.shape[1], use_norm=True, top_k=top_k, use_gpu=False)
  print("1---")
  vectorized_norm = np.sum(vectorized_plots**2, axis=1).reshape((-1,1))
  
  def recommend(query):
    try:
      idx = retriever.predict(vectorized_plots, 
                              vectorized_plots[movie_names.index(query)], 
                              vectorized_norm)[0][1:]
      for i in idx:
        print(names[i])
    except ValueError:
      print("{} not found in movie db. Suggestions:")
      for i, name in enumerate(movie_names):
        if query.lower() in name.lower():
          print(i, name)
          
  return recommend

In [0]:
#recommend = buildMovieRecommender(title, X_vect)
#recommend("global economy")

In [0]:
q="how can ECB help to instill confidence in banks?"
q_vect = bert_vectorizer(q, verbose=True)


48/48 [==============================] - 0s 1ms/step


In [0]:
q_vect

array([[ 1.22066304e-01, -5.16647756e-01,  1.16507754e-01,
         4.74443614e-01,  7.64028907e-01, -4.95769083e-01,
        -1.69877216e-01,  3.29991251e-01, -2.88894415e-01,
        -3.64405155e-01,  3.25473577e-01, -2.62071073e-01,
         1.33199915e-01,  2.49008775e-01, -3.50371808e-01,
        -2.02133298e-01,  9.95713845e-02, -2.54450925e-02,
         4.06343222e-01, -8.09697956e-02, -5.21905541e-01,
         5.44043541e-01,  1.14256859e-01,  3.92054260e-01,
         8.38345960e-02,  8.68858844e-02,  6.34426415e-01,
        -1.48037240e-01,  8.80607441e-02, -2.58684289e-02,
         3.78071487e-01,  2.74111748e-01,  1.54573675e-02,
        -2.45931849e-01, -4.42773998e-02, -1.55441567e-01,
        -2.55708545e-01, -2.76611894e-01, -2.88986206e-01,
         1.76114053e-01, -4.32723731e-01, -1.18127063e-01,
        -1.14535354e-01,  3.44993830e-01, -3.33465576e-01,
        -2.32168317e-01, -1.17062703e-01,  5.31404726e-02,
         3.65995258e-01, -4.45680767e-01, -6.18691385e-0

In [0]:
def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    
    cosine = scipy.spatial.distance.cosine(s1, s2)
    #print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')
    return round((1-cosine)*100,2)
    
def euclidean_distance_wordembedding_method(s1, s2):
    import scipy
    
    euclidean = scipy.spatial.distance.euclidean(s1, s2)
    #print('Word Embedding method with a euclidean distance btwn two sentences ',round((1-euclidean)*100,2),'%')
    return round((euclidean),2)


In [0]:
def find_similar_docs_cos(X_vect,q):
  q_vect = bert_vectorizer(q, verbose=True)
  dist={}
  for ind,val in enumerate(X_vect):
    dist[ind]= cosine_distance_wordembedding_method(val,q_vect)

  dist_sort=sorted(dist.items(), key = lambda kv:(kv[1]),reverse=True)
  return dist_sort


def find_similar_docs_euc(X_vect,q):
  q_vect = bert_vectorizer(q, verbose=True)
  dist={}
  for ind,val in enumerate(X_vect):
    dist[ind]= euclidean_distance_wordembedding_method(val,q_vect)
  dist_sort=sorted(dist.items(), key = lambda kv:(kv[1]),reverse=False)
  return dist_sort

print(find_similar_docs_cos(X_vect,"what did the Dutch central bank governor tell?") )


46/46 [==============================] - 0s 1ms/step
[(2, 71.49), (0, 70.92), (4, 70.52), (1, 70.14), (3, 69.67), (5, 63.61), (6, 61.99)]


In [0]:
#q="how much money did Italian banks take from central bank funds"  #txt5  c-txt4  wrong
#q="which banks used the first LTRO to pursue Sarkozy trade"  #txt5  c-txt5  correct
#q="what is sarkozy trade" #txt5  c-txt1  wrong
#q="will European Central Bank extend loan term?" #txt2  c=txt2  correct
#q="What does the physics engine allow for?" #phys  c= phys  correct
#q="when did ECB introduce 12-month liquidity tenders?"#txt2  c=txt4 wrong
#q="what did the official euro zone sources announce" #txt4  c=txt1  wrong
#q="which country needs to sell 45 billion euros bonds"  #txt5  c=txt2 wrong
#q="what do the roll-over accounts for"  #txt3  c=txt3 correct
#q= "what does ECB data calculations tell about excess liquidity in the system" #txt3 c=txt3  correct
#q= "what does ECB data tell about excess liquidity in the system" #txt3 c=txt3  correct
#q="for what are the investors observing euro zone money market"  #txt3  c=txt1  wrong
#q=" can monetary policy solve tension in finance markets ?"  #txt1  c=txt1 correct
#q="""what is the meaning of combinatorial explosion"""#txt6  c= txt6  correct
#q="when did ECB first introduce extra-long 12-month liquidity tenders"  #txt2  c=txt4 wrong
#q="what banks are major holders country's debt" #txt4  c=txt4 correct
print(find_similar_docs_cos(X_vect,q))
print(find_similar_docs_euc(X_vect,q))


66/66 [==============================] - 0s 927us/step
[(3, 80.3), (4, 79.78), (1, 79.75), (2, 79.52), (0, 73.45), (5, 67.47), (6, 63.41)]
66/66 [==============================] - 0s 835us/step
[(3, 8.03), (2, 8.07), (4, 8.08), (1, 8.09), (0, 9.06), (5, 10.42), (6, 10.65)]


In [0]:

s1="children playing in park"
s2="kids running between the trees"
s3="traffic is bad today"
d1 = bert_vectorizer(s2, verbose=True)
d2 = bert_vectorizer(s3, verbose=True)
cosine_distance_wordembedding_method(d1,d2)

# 01     02    12
#85.22  67.6     69.01

#eu 7.8  10.6    10.9

20/20 [==============================] - 0s 2ms/step


70.08